# TMP91FW27UG Bootloader Attack




## Talking to Bootloader

You should have done the bootloader setup in Day 1. We'll be using the same setup, but with two differences:

1) We'll be doing a glitch attack.

2) We'll be targetting a different function.

## Finding Glitch Settings

As in earlier labs, we'll need to find suitable settings. This can be done with voltage or clock glitching, but this lab uses clock glitching. The following is roughly what is required of you:

1. Find a suitable glitch function.
2. Play with settings for the glitch.

Let's get to it!

### Clock Glitching

Unlike previous labs, we'll be using clock glitching for this one, as it seems to work pretty well on this target. This doesn't change too much, as you'll still need to find a set of widths/offsets that work for your device. The setup will be a bit different (mostly using and different output setting and putting the glitch on the clock pin instead of the glitch transistors). One thing that will be different is that you won't have to stop part way to reduce the width range. You'll also probably see fewer resets than with voltage glitching.

### Finding a Suitable Function

In previous labs, we used a loop to calibrate our glitch. This obviously won't be available on this target. Instead, to calibrate our glitch, we'd like to use one of the bootloader operations. Ideally, this operation will allow us to identify if the device did one of the following things:

1. Reset
2. Worked Normally (no effect)
3. Was impacted but a glitch (success!)

In addition, we don't want this operation to be very timing-dependant (i.e. you don't want to have to scan `ext_offset` very precisely).

Take a look at the various functions in the bootloader and decide on what to attack. You'll need the op-code for the function.

### Setting up Normal Operation

Setup some code to operate the function normally. We'll use this as a bit of an oracle for understanding the normal output.

In [ ]:
PLATFORM = ""
%run ../Setup_Scripts/Setup_Generic.ipynb

scope.clock.clkgen_freq = 25E6 #25 MHz on examples
target.baud = 9600 #9600 baud rate
scope.io.tio1 = "serial_tx"
scope.io.tio2 = "serial_rx"

In [ ]:
def reset_target():
    scope.io.pdic = False
    scope.io.nrst = False
    time.sleep(0.05)
    scope.io.nrst = True
    time.sleep(0.05)
    target.flush()    

In [ ]:
def calc_checksum(r):
    s = 0
    for c in r:
        s += c
    cs = 0x100 - (s & 0xff)
    return cs

def tx_rx(cmd, expectedlen=0, rxlen=100):
    target.write(cmd)
    response = target.read(rxlen)
    if expectedlen:
        if len(response) != expectedlen:
            raise IOError("Unexpected response length %d (data: %s)"%(len(response), str(response)))

    responsehex = [ord(c) for c in response]
    return response, responsehex

You can use the following code which syncronizes to the device as an example:

In [ ]:
reset_target()
response, responsehex = tx_rx(b"\x86", 1, 1)
if responsehex[0] != 0x86:
    raise IOError("Sync Error")
    
# ###################
# Add your code here to do the bootloader operation you've selected (Code Block 1)
# ###################
raise NotImplementedError("Add your code here, and delete this.")

## Glitching 

In [ ]:
gc = cw.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset"])
gc.display_stats()
gc.glitch_plot(plotdots={"success":"+g", "reset":"xr", "normal":None})

In [ ]:
gc.set_range("width", -5, 5)
gc.set_range("offset", -5, 5)
gc.set_global_step([5.0, 2.5])

Next we'll setup the glitch module. You could use `scope.cglitch_setup()`, but we've got the full setup here as you may not have seen clock glitching on ChipWhisperer before.

In [ ]:
#Basic setup
if scope._is_husky:
    scope.glitch.enabled = True

scope.glitch.clk_src = "pll" # set glitch input clock
scope.glitch.output = "clock_xor" # glitch_out = clk ^ glitch
scope.glitch.trigger_src = "ext_single" # glitch only after scope.arm() called

scope.io.hs2 = "glitch"  # output glitch_out on the clock line
scope.trigger.triggers = "tio2"
print(scope.glitch)

Fill in your communication code below. Feel free to stop this loop early if you've got a good set of settings to work with.

In [ ]:
from tqdm.notebook import trange
import struct

scope.glitch.ext_offset = 100000

# width and offset numbers have a very different meaning for Husky vs Lite/Pro;
# see help(scope.glitch) for details
if scope._is_husky:  
    gc.set_range("width", 0, 400)
    gc.set_range("offset", 400, 1400)

    gc.set_global_step([5])
    scope.adc.lo_gain_errors_disabled = True
    scope.adc.clip_errors_disabled = True
else:
    gc.set_range("width", 0, 48)
    gc.set_range("offset", -48, 48)
    gc.set_global_step([8, 4, 2, 1, 0.4])
    
scope.glitch.repeat = 5

scope.adc.timeout = 0.1

broken = False
for glitch_setting in gc.glitch_values():
    reset_target()
    scope.glitch.offset = glitch_setting[1]
    scope.glitch.width = glitch_setting[0]
    
    reset_target()
    target.flush()
    response, responsehex = tx_rx(b"\x86", 1, 1)
    if responsehex[0] != 0x86:
        raise IOError("Sync Error")
    
    scope.arm()

    #Do glitch loop
    raise NotImplementedError("Change the following to be the selected op-code")
    target.write(b"\x??")

    ret = scope.capture()
    
    loff = scope.glitch.offset
    lwid = scope.glitch.width

    if ret:
        print('Timeout - no trigger')
        gc.add("reset")

        #Device is slow to boot?
        reset_target()
    else:
        # ###################
        # Add your code here to do the bootloader operation you've selected (Code Block 1)
        # ###################
        raise NotImplementedError("Add your code here, and delete this.")
        
        
        if len(response) == 0:
            gc.add("reset")
        else:
            raise NotImplementedError("Change the following to be the known-good response")
            if  response != [32, 250, 165, 97]:
                broken = True
                gc.add("success")
                print(response)
                print(loff)
                print(lwid)
                print("🐙", end="")
            else:
                gc.add("normal")

print("Done glitching")

## RAM Attack

The final stage is to actually try to send the known password (discovered with the SPA attack), followed by the glitch. We'll leave this one more open-ended. Start with the above loop and modify from there to work with your known-good settings.